In [1]:
import os
from pymongo import MongoClient
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import re
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import getpass
from graph_tool.all import *


class FriendListManager():
    def __init__(self):
        self.driver = self.login()
        self.client = MongoClient()
        self.scraperdb = self.client.scraper_db
        self.scraperData = self.scraperdb.scraperData

    def login(self,username=None, password=None):
        if not username:
            username =raw_input("enter your username:")
            password = getpass.getpass()
        if not password:
            password = getpass.getpass()
        chromeDriver = '/Users/Jai/Downloads/chromedriver'
        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.managed_default_content_settings.notifications": 2}
        chrome_options.add_experimental_option("prefs", prefs)

        os.environ["webdriver.chrome.driver"] = chromeDriver
        driver = webdriver.Chrome(chromeDriver, chrome_options=chrome_options)

        driver.get("https://web.facebook.com/")
        assert 'Facebook' in driver.title
        email_field = driver.find_element_by_id("email")
        pass_field = driver.find_element_by_id("pass")

        email_field.send_keys(username)
        pass_field.send_keys(password)
        pass_field.send_keys(Keys.RETURN)
        time.sleep(2)
        return driver

    def getLikesFromPosts(self,target):
        # open the persons profile
        pageURL = str()
        if re.match(r'https?\:\/\/www\.facebook\.com\/[^?]*\?',target):
            pageURL = target
        else:
            pageURL = "https://web.facebook.com/" + target
        self.driver.get(pageURL)
        # TODO: replace with code that ensures we are at the bottom of the page
        for i in range(2):
            for i in range(2):
                ActionChains(self.driver).key_down(Keys.DOWN).perform()
            time.sleep(0.1)
        try:
            coverPhotoObj = self.driver.find_element_by_xpath('//*[@id="fbCoverImageContainer"]')
            coverPhotoObj.click()
        except NoSuchElementException:
            print "Cover photo not found"
        time.sleep(5)
        friends = set()
        reactionPageURLs = set()
        CoverObjectReactionElements = set()
        allCoverLinks = self.driver.find_elements_by_tag_name('a')
        for x in allCoverLinks:
            if '/ufi/reaction/profile/browser/' in str(x.get_attribute('href')):
                CoverObjectReactionElements.add(x.get_attribute('href'))
        reactionPageURLs = reactionPageURLs.union(CoverObjectReactionElements)
        try:
            closeButtonObj = self.driver.find_element_by_xpath('//*[@id="photos_snowlift"]/div[2]/div/a')
            closeButtonObj.click()
        except NoSuchElementException:
            print "DP not found"
        time.sleep(1)
        DPObj = self.driver.find_element_by_class_name('profilePic')
        DPObj.click()
        time.sleep(5)
        DPReactionElements = set()
        allDPLinks = self.driver.find_elements_by_tag_name('a')
        for x in allDPLinks:
            if '/ufi/reaction/profile/browser/' in str(x.get_attribute('href')):
                DPReactionElements.add(x.get_attribute('href'))
        reactionPageURLs = reactionPageURLs.union(DPReactionElements)

        # print "links found:"
        # for URL in reactionPageURLs:
        #     print URL
        for element in reactionPageURLs:
            link = element
            print "exploring " + str(link)
            friendsFoundFromLink = self.getLikesFromReactionPage(link)
            friends = friends.union(friendsFoundFromLink)
        return friends

    def getLikesFromReactionPage(self,likelistURL):
        self.driver.get(likelistURL)
        more = True
        while more:
            try:
                time.sleep(5)
                seemorebutton = self.driver.find_element_by_xpath('//*[@id="reaction_profile_pager1"]/div')
                seemorebutton.click()
            except NoSuchElementException:
                more = False
        friendlinks = self.driver.find_elements_by_xpath(
            '//*[@id="reaction_profile_browser1"]/li/div/div/div/div[2]/div[2]/div/a')
        result = set()
        for linkelement in friendlinks:
            # result.add(linkelement.get_attribute("href"))
            link = linkelement.get_attribute("href")
            name = linkelement.text
            idsearch = re.search(r'(https?\:\/\/(www\.)?(web\.)?facebook\.com\/)([^?]*)\?',link)
            if idsearch:
                #print "adding " + idsearch.group(4)
                result.add((idsearch.group(4),name))
        return result

    def hasFriendListVisible(self,target):
        self.driver.get("https://web.facebook.com/" + target + '/friends')
        time.sleep(2)
        types_visible = self.driver.find_elements_by_class_name('_3sz')
        for friendType in types_visible:
            if friendType.text == 'All Friends':
                return True
        return False

    def getFriendListFromFriendsPage(self,target):
        self.driver.get("https://web.facebook.com/" + target + '/friends')
        lastHeight = self.driver.execute_script("return document.body.scrollHeight")
        while True:
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            newHeight = self.driver.execute_script("return document.body.scrollHeight")
            if newHeight == lastHeight:
                break
            lastHeight = newHeight
        friendsOfTarget = self.driver.find_elements_by_class_name('fwb')

        friends = list()
        friends_names = list()
        for element in friendsOfTarget:
            try:
                friend_link = element.find_element_by_tag_name('a')
                if friend_link and friend_link.get_attribute('href') and re.match(
                    r'(https?\:\/\/(www\.)?(web\.)?facebook\.com\/)([^?]*)\?', friend_link.get_attribute('href')):
                    friends.append(friend_link.get_attribute('href'))
                    friends_names.append(friend_link.text)
                    #print friend_link.get_attribute('href')
            except NoSuchElementException:
                continue
        result = set()
        zipped = zip(friends,friends_names)
        for (link,name) in zipped:
            idsearch = re.search(r'(https?\:\/\/(www\.)?(web\.)?facebook\.com\/)([^?]*)\?',link)
            if idsearch:
                newelement= (idsearch.group(4),name)
                result.add(newelement)
        return result

    def getFriendListHelper(self,target):
        if target =='profile.php':
            return set()
        if self.hasFriendListVisible(target):
            return self.getFriendListFromFriendsPage(target)
        else:
            return self.getLikesFromPosts(target)

    def getFriendList(self,target):
        friendListEntry = self.scraperData.find_one({"user":target})
        if friendListEntry == None:
            friendSet = self.getFriendListHelper(target)
            friendList = list(friendSet)
            entry = {"user" : target, "friendList":friendList}
            self.scraperData.insert_one(entry)
            return friendList
        else:
            return friendListEntry["friendList"]

    def storeFriendList(self,root, FriendList):
        entry = {"user": root , "friendList": FriendList}
        self.scraperData.insert_one(entry)
        return True

In [3]:
class graphExplorer:
    def __init__(self):
        self.flm = FriendListManager()
        self.g = None
        
    def genFriendGraph(self,target, friendList):
        '''
        To generate the graph for a given root node and his/her friend list
        :param target: the root node as (fb-id , name)
        :param friendList: a list of tuples of (link,name)
        :return: the graph object
        '''
        self.g = Graph()
        root = self.g.add_vertex()
        self.g.gp.idtovert = self.g.new_gp("python::object")
        self.g.gp.idtovert = dict()
        self.g.gp.idtovert[target[0]]=0
        self.g.vp.vertexText = self.g.new_vp("string")
        self.g.vp.vertexText[root] = target[1]
        self.g.vp.id = self.g.new_vp("string")
        for friend in friendList:
            friendVertex = self.g.add_vertex()
            self.g.gp.idtovert[friend[0]] = self.g.vertex_index[friendVertex]
            self.g.vp.vertexText[friendVertex] = friend[1]
            #print "making node with id " + friend[0]
            self.g.vp.id[friendVertex] = friend[0]
            if not self.g.edge(root,friendVertex):
                self.g.add_edge(root,friendVertex)
    
    
    def addNewFriends(self,target, friends):
        '''
        :param target: the person I'm adding to the graph - (fb-id, name)
        :param friends: A list of (link,name) tuples of the friends of the person in consideration
        '''
        if target[0] in self.g.gp.idtovert:
            targetVertex = self.g.vertex(self.g.gp.idtovert[target[0]])
        else:
            targetVertex = self.g.add_vertex()
        self.g.gp.idtovert[target[0]]=self.g.vertex_index[targetVertex]
        self.g.vp.vertexText[targetVertex] = target[1]
        for friend in friends:
            if friend[0] not in self.g.gp.idtovert:
                friendVertex = self.g.add_vertex()
            else:
                friendVertex = self.g.vertex(self.g.gp.idtovert[friend[0]])
            self.g.gp.idtovert[friend[0]] = self.g.vertex_index[friendVertex]
            self.g.vp.id[friendVertex] = friend[0]
            self.g.vp.vertexText[friendVertex] = friend[1]
            #print "adding edge to " + friend[1]
            if not self.g.edge(targetVertex,friendVertex):
                self.g.add_edge(targetVertex,friendVertex)
    
    def interactiveWindow(self):
        print " Input as follows \n \
        1 for importing a new person into the graph \n \
        2 for exploring an existing person in the graph \n \
        3 for displaying the graph inline \n \
        4 for displaying the graph in a separate window \n \
        5 for exiting the interactive window"
        recall= True
        choice= input()
        if choice == 1:
            newid = raw_input("enter the id of the person you want to explore")
            newname = raw_input("enter the name of the person")
            if self.g:
                self.addNewFriends((newid,newname),self.flm.getFriendList(newid))
            else:
                self.genFriendGraph((newid,newname),self.flm.getFriendList(newid))
        elif choice==2:
            if not self.g:
                print "the graph is empty right now, please choose another option!"
                self.interactiveWindow()
            explorename = raw_input("enter the name of the person you want to explore")
            exploreids = [self.g.vp.id[x] for x in self.g.vertices() if explorename.lower() in str(self.g.vp.vertexText[x]).lower()]
            print "choose from one of the following:"
            print exploreids
            exploreid = raw_input()
            self.addNewFriends((exploreid,explorename),self.flm.getFriendList(exploreid))
        elif choice==3:
            graph_draw(self.g,inline=True,vertex_text= self.g.vp.vertexText,vertex_font_size=6)
        elif choice == 4:
            graph_draw(self.g,inline=False,vertex_text= self.g.vp.vertexText,vertex_font_size=6)
        elif choice == 5:
            recall=False
        
        if recall:
            self.interactiveWindow()


In [ ]:
ge = graphExplorer()
ge.interactiveWindow()

In [ ]:
sahil arora
dhruv jain
charu anand